In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import h_key

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/hotel.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

### API Calls

In [2]:
#Ryan API

In [3]:
#Nicole API

In [4]:
# Create aiport dataframe
city_df = pd.read_csv('./output_data/City-Airport_Output.csv')
city_df.dropna(inplace = True) 
city_df.head()


,Unnamed: 0,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,0,New York City,New York,40.712775,-74.005973,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.2
1,1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.6
2,2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6
3,3,Houston,Texas,29.760427,-95.369803,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",3.7
4,4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3


In [5]:
#Kayla API

import requests


url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"

querystring = {"lat":"37.788719679657554","lon":"-122.40057774847898","checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

headers = {
    'x-rapidapi-key': h_key,
    'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

response.json()

response = response.json()


{"result":"OK","data":{"body":{"header":"22 Anthony St, San Francisco, CA 94105, USA","query":{"destination":{"id":"1493604","value":"22 Anthony St, San Francisco, CA 94105, USA","resolvedLocation":"GEO_LOCATION:22 Anthony St, San Francisco, CA 94105, USA|37.78880310058594|-122.40035247802734:GEOCODE:UNKNOWN"}},"searchResults":{"totalCount":412,"results":[{"id":693927,"name":"Hotel Ikon","starRating":2.5,"urls":{},"address":{"streetAddress":"325 Sutter Street","extendedAddress":"","locality":"San Francisco","postalCode":"94108","region":"CA","countryName":"United States","countryCode":"us","obfuscate":false},"welcomeRewards":{"collect":true},"guestReviews":{"unformattedRating":3.3,"rating":"3.3","total":188,"scale":5,"badge":"good","badgeText":"Good"},"landmarks":[{"label":"22 Anthony St, San Francisco, CA 94105, USA","distance":"0.3 miles"},{"label":"San Francisco","distance":"0.2 miles"}],"geoBullets":[],"ratePlan":{"price":{"current":"$73","exactCurrent":73.26,"old":"$99","info":"ni

In [6]:
response["data"]

{'body': {'header': '22 Anthony St, San Francisco, CA 94105, USA',
  'query': {'destination': {'id': '1493604',
    'value': '22 Anthony St, San Francisco, CA 94105, USA',
    'resolvedLocation': 'GEO_LOCATION:22 Anthony St, San Francisco, CA 94105, USA|37.78880310058594|-122.40035247802734:GEOCODE:UNKNOWN'}},
  'searchResults': {'totalCount': 412,
   'results': [{'id': 693927,
     'name': 'Hotel Ikon',
     'starRating': 2.5,
     'urls': {},
     'address': {'streetAddress': '325 Sutter Street',
      'extendedAddress': '',
      'locality': 'San Francisco',
      'postalCode': '94108',
      'region': 'CA',
      'countryName': 'United States',
      'countryCode': 'us',
      'obfuscate': False},
     'welcomeRewards': {'collect': True},
     'guestReviews': {'unformattedRating': 3.3,
      'rating': '3.3',
      'total': 188,
      'scale': 5,
      'badge': 'good',
      'badgeText': 'Good'},
     'landmarks': [{'label': '22 Anthony St, San Francisco, CA 94105, USA',
       'dis

In [7]:
counter = 0
cities = []
results=[]

    

#For loop
for city,item in city_df.iterrows():
    counter=counter+1  
    #print(item)
    try:
        url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"
        #print(item["Lat"],item["Lng"]) 
        querystring = {"lat":item["Lat"],"lon":item["Lng"],"checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

        headers = {
        'x-rapidapi-key': h_key,
        'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        response = response.json()
    

        nh = len(response['data']['body']['searchResults']['results'])

        for i in range(nh):

            Hotel = response['data']['body']['searchResults']['results'][i]['name']
            Rating = response['data']['body']['searchResults']['results'][i]['starRating']
            City = response['data']['body']['searchResults']['results'][i]['address']['locality']
            State = response['data']['body']['searchResults']['results'][i]['address']['region']
            Country =response['data']['body']['searchResults']['results'][i]['address']['countryCode']
            Stay_Rate = response['data']['body']['searchResults']['results'][i]['ratePlan']['price']['exactCurrent']
            
            results.append([Hotel, Rating,City, State, Country,Stay_Rate])
            
    except:
        
        print("Hotel Data Not Found")
   
               
print(results)


Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data Not Found
Hotel Data No

In [8]:
hotel_df = pd.DataFrame(results,columns=["Hotel","Rating","City", "State", "Country","Price"])
hotel_df

hotel_df.to_csv('HOTEL_API.csv', index=False)